1\. Write a function that converts number representation (bin<->dec<->hex)

In [10]:
def func_bin_dec_hex():
 dec_str = 23
 print('decimal input:', dec_str)
    
 s_bin = bin(dec_str)  
 print('binary equivalent:',s_bin)
 
 s_hex = hex(dec_str)
 print('hexadecimal equivalent:',s_hex)

func_bin_dec_hex()

decimal input: 23
binary equivalent: 0b10111
hexadecimal equivalent: 0x17


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [29]:
import math

def func():
#    word ='11000000101100000000000000000000'
    word= '00000011111000000000000000000000'
    
    word_len = len(word)
    print('word length:',word_len)
   # print(int(word))
   
    #for mantissa
    floating_in_bits = []
    
    for i in range(len(word) - 9):
        floating_in_bits.append(word[31-i])
        
    floating_in_bits2 = list(reversed(floating_in_bits))
    print("floatiing bits:",floating_in_bits2)
    print(len(floating_in_bits2))
    
    manti = 0
    for i in range(len(floating_in_bits2)):
         manti = (int(floating_in_bits2[i]) * pow(2,-i-1)) + manti
    print("mantissa:",manti)
    
    # exponential
    
    exp_in_bits = []
    for j in range(23,31):
        exp_in_bits.append(word[31-j])
    exp_in_bits2 = list(reversed(exp_in_bits))
    print(exp_in_bits2)
    
    exp = 0
    for i in range(8):
        exp = (int(exp_in_bits2[i])) * pow(2,7-i)+ exp
       # print(exp_in_bits2[i])
    print(len(exp_in_bits2))
    print("exponential:",exp) 
    
    #sign
    
    print("sign bit:",word[0])
    if(word[0] == '0'):
        sign = +1
    if(word[0] == '1'):
        sign = -1
    print("sign:", int(sign))
    
    # putting together
    x_float = pow(-1,sign) * (1 + manti) * pow(2, exp - 127)
    print("single precision floating point value:",x_float)
    
func()

word length: 32
floatiing bits: ['1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
23
mantissa: 0.75
['0', '0', '0', '0', '0', '1', '1', '1']
8
exponential: 7
sign bit: 0
sign: 1
single precision floating point value: -1.316553672920962e-36


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [82]:

def func_overflow():
    var1 = 1
    var_temp = 1
    while (var1 != float("inf")):
        var_temp = var1
        var1 = var1 * 2
    return var_temp
print(func_overflow())

def func_underflow():
    var2 = 1
    var_temp = 1
    while(var2 != 0):
        var_temp = var2
        var2 = var2 / 2
    return var_temp
print(func_underflow())

KeyboardInterrupt: 

4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [31]:
import math

# part a
def quad_form(a,b,c):
    x1 = (-b + math.sqrt((b**2) - 4 * a * c))/ (2 * a)
    x2 = (-b - math.sqrt((b**2) - 4 * a * c))/ (2 * a)
    return x1,x2
x = 0.001
y = 1000
z = 0.001
print(quad_form(x,y,z))

# part b
def func2(a,b,c):
    x1 = (2*c)/(-b + math.sqrt(b**2 - 4 * a * c))
    x2 = (2*c)/(-b - math.sqrt(b**2 - 4 * a * c))
    return x1, x2

a = 0.001
b = 1000
c = 0.001
print(func2(a,b,c))

#explanation in partb (why? part)
#adding subtracting '-b' and the 'sqrt() term with different orders of magnitude
#can cause loss of accuracy. In fact partb is called Citardauq formula

#partc
def func3(a,b,c):
    epsilon = 10e-4
    if( (b - (math.sqrt(b**2 - 4 * a * c))) <= epsilon ):
        quad_form(a,b,c)
    else:
        func2(a,b,c)
print(func3(1,100,1))
        

(-9.999894245993346e-07, -999999.999999)
(-1000010.5755125057, -1.000000000001e-06)
None


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [28]:
def func(x):
    return x * (x-1)

#part a,b mixed
def deri_func(x):
    i = 2
    while(i <= 14) :
        delta = pow(10,-i)
        deriv = (func(x+delta) - (func(x)))/delta
        print(deriv)
        i = i + 2     
deri_func(0.5)

0.009999999999998899
9.999999994736442e-05
1.0000056338554941e-06
1.1102230246251565e-08
0.0
0.0
0.0


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [81]:
#part a and part b combined
import math
import time 

N = 1400000
I = 0
h = 2/N
a = -1
b = 1
x = -1

time1 = time.time()
for i in range (1,N):
    I = I + h * (math.sqrt(1 - x*x))
    x = x + h
time2 = time.time()
t2  = time2 - time1
print("calculated value:",I)

true_value = (math.pi)/2
print("true value:",true_value)

accuracy = abs(I - true_value)
print("accuracy error:",(accuracy/true_value)*100,"%")

t = time2 -time1
print("time taken for N = 100 is:",t)

calculated value: 1.5707963233580307
true value: 1.5707963267948966
accuracy error: 2.18797678881542e-07 %
time taken for N = 100 is: 1.0322527885437012
